i have built an basic question-answering chatbot that can answer queries about college policies (like attendance rules) by extracting and searching real content from online pages, "without using any paid APIs like OpenAI" By adding more website links which contains college information we can make it better.

i have installed langchain,FAISS,Sentence Trasformers, and tools for web scraping i have used RecursiveURLLoader from specific Quora pages related to the college.

spliting the content into smaller chunks and then converting into vector embeddings locally, after converting into vectors stored the data into vector base(data base)so that user's can access information for the asked questions in no time instead of going through websites

In [1]:
!pip install sentence-transformers faiss-cpu beautifulsoup4 requests langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 62.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [3]:
pip install langchain

In [22]:
!pip install langchain faiss-cpu sentence-transformers
from langchain.document_loaders import RecursiveUrlLoader
from sentence_transformers import SentenceTransformer

In [24]:
!pip install sentence-transformers langchain faiss-cpu

In [3]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

In [19]:
urls = [
    "https://www.quora.com/What-is-the-minimum-attendance-required-for-exams-in-VIT",
    "https://www.quora.com/How-are-the-college-timings-at-VIT-Vellore"
]

all_docs = []
for url in urls:
    loader = RecursiveUrlLoader(url=url, max_depth=2, use_async=False)
    all_docs.extend(loader.load())

print("Loaded", len(all_docs), "documents.")
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = splitter.split_documents(all_docs)
print("Total chunks:", len(split_docs))

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-MiniLM-L3-v2")

vectorstore = FAISS.from_documents(split_docs, embedding_model)

Loaded 2 documents.
Total chunks: 3024


In [13]:
vectorstore.similarity_search("What is the attendance policy?",k=1)

[Document(id='1af4f771-0ea2-4dd4-ae06-7e23a27c8064', metadata={'source': 'https://www.quora.com/What-is-the-minimum-attendance-required-for-exams-in-VIT', 'content_type': 'text/html; charset=utf-8', 'title': 'What is the minimum attendance required for exams in VIT? - Quora', 'description': 'You have to have at least 75% attendance in each subject before any set of test namely CAT1, CAT2, FAT tests. Please note the attendance is counted up to 1 day prior to the last instructional day before the test. Although if your CGPA is above 9 y...', 'language': 'en'}, page_content='there any exceptions to the attendance policy for special circumstances?\\",\\"What are the consequences of not meeting the attendance requirement?\\",\\"How is attendance monitored at DAIICT during lectures and')]

In [5]:
def ask_bot(question, k=1):
    results = vectorstore.similarity_search(question, k=k)
    return "\n\n".join([res.page_content for res in results])


In [14]:
ask_bot("What is the minimum attendance required for exams in VIT?")

'#VIT", "upvoteCount": 0, "url": "https://www.quora.com/What-is-the-minimum-attendance-required-for-exams-in-VIT?top_ans=1477743858770584"}], "text": "What is the minimum attendance required for exams in VIT?", "upvoteCount": 1}}'